In [ ]:
import pathlib

source_root = pathlib.Path(r'D:\TEMP\flattened')
output_folder = pathlib.Path(r'D:\TEMP\output')

In [ ]:
from takeout_tools.matching import pair_single_folder

paired, unmatched_media, unmatched_metadata = pair_single_folder(source_root, ignore_errors=True, compare_ext_name=True)

In [ ]:
unmatched_media

In [ ]:
unmatched_metadata

In [ ]:
from takeout_tools import MediaHandler

MediaHandler.list_handlers()

In [ ]:
from takeout_tools.matching import MediaVersion

# select first matched version
def prefer_version(files: list[dict], version) -> tuple[list[dict], list[dict]]:
    selected, others = [], []
    for f_info in files:
        if f_info['version'] == version and not selected:
            selected.append(f_info)
        else:
            others.append(f_info)
    return selected, others

In [ ]:
from functools import partial

output_folder.mkdir(exist_ok=True)
output_not_selected_folder = output_folder / 'not_selected'
output_not_selected_folder.mkdir(exist_ok=True)

for metadata, files in paired:
    print()
    selected, others = prefer_version(files, MediaVersion.ORIGINAL)
    
    for media_info in selected:
        MediaHandler.auto(
            media_info,
            metadata,
            target_folder=output_folder,
            allow_fallback=True,
        )
    
    for media_info in others:
        MediaHandler.auto(
            media_info,
            metadata,
            target_folder=output_not_selected_folder,
            allow_fallback=True,
        )


In [ ]:
unmatched_output_folder = output_folder / 'unmatched'
unmatched_files = [d['media_path'] for d in unmatched_media] + [d['metadata_path'] for d in unmatched_metadata]

In [ ]:
unmatched_files

In [ ]:
from functools import partial
import shutil

unmatched_output_folder.mkdir(exist_ok=True)
_ = list(map(partial(shutil.copy, dst=unmatched_output_folder), unmatched_files))